In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.layers import Dense, Embedding, Input, LSTM, GRU, SimpleRNN
from keras.callbacks import Callback
from keras.models import Sequential
from tensorflow import keras
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np
import csv

## **Code without Wandb Sweeps**

---

In [ ]:
#Data Path
train_path = '/content/drive/MyDrive/DL_Assignment 3/lexicons/Train.tsv'
test_path = '/content/drive/MyDrive/DL_Assignment 3/lexicons/Test.tsv'
val_path = '/content/drive/MyDrive/DL_Assignment 3/lexicons/Val.tsv'

train = pd.read_csv(train_path, delimiter='\t')
val = pd.read_csv(val_path, delimiter='\t')
test = pd.read_csv(test_path, delimiter='\t')

'''To segregate most commonly occurring spellings of 25000 unique words out of 44204 words in Training data
and most commonly occurring spellings of 2500 unique words out of 4358 words in Validation data.'''
# train = train.sort_values('Count').drop_duplicates('Hindi', keep='last')
# val = val.sort_values('Count').drop_duplicates('Hindi', keep='last')
# test = test.sort_values('Count').drop_duplicates('Hindi', keep='last')
# train = train[['Hindi', 'English']].copy()
# val = val[['Hindi', 'English']].copy()
# test = test[['Hindi', 'English']].copy()

print(len(train))
print(len(val))
print(len(test))
print(train)
print(val)
print(test)

44204
4358
4502
           Hindi     English  Count
0             अं          an      3
1        अंकगणित    ankganit      3
2           अंकल       uncle      4
3          अंकुर       ankur      4
4         अंकुरण     ankuran      3
...          ...         ...    ...
44199  ह्वेनसांग  hiuentsang      1
44200  ह्वेनसांग  hsuantsang      1
44201  ह्वेनसांग    hyensang      1
44202  ह्वेनसांग    xuanzang      1
44203          ॐ          om      3

[44204 rows x 3 columns]
        Hindi  English  Count
0        अंकन    ankan      3
1      अंगकोर   angkor      3
2      अंगिरा   angira      3
3      अंगीठी  angithi      3
4     अंग्रेज   angrej      3
...       ...      ...    ...
4353   होल्डर   holder      3
4354     होशो    hosho      3
4355   ह्यूअर     huar      1
4356   ह्यूअर    hyuar      1
4357   ह्यूअर    hyuer      1

[4358 rows x 3 columns]
          Hindi       English  Count
0           अंक           ank      5
1           अंक          anka      1
2         अंकित         ankit 

In [ ]:
#Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for index, line in train.iterrows():
    input_text, target_text = line['English'], line['Hindi']
    input_text, target_text = str(input_text), str(target_text)
    target_text = " " + target_text + " "
    input_text = " " + input_text + " "
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])+1
max_decoder_seq_length = max([len(txt) for txt in target_texts])+1

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
print(target_characters)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t]  =  input_token_index[char]
    encoder_input_data[i, t + 1:] = input_token_index[" "]
    for t, char in enumerate(target_text):
        decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1:] = target_token_index[" "]
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 44204
Number of unique input tokens: 35
Number of unique output tokens: 64
Max sequence length for inputs: 23
Max sequence length for outputs: 22
[' ', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॐ']


In [ ]:
#Vectorize the data.
input_texts_val = []
target_texts_val = []
input_characters_val = set()
target_characters_val = set()
for index, line in val.iterrows():
    input_text_val, target_text_val = line['English'], line['Hindi']
    input_text_val, target_text_val = str(input_text_val), str(target_text_val)
    target_text_val = " " + target_text_val + " "
    input_text_val = " " + input_text_val + " "
    input_texts_val.append(input_text_val)
    target_texts_val.append(target_text_val)
    for char in input_text_val:
        if char not in input_characters_val:
            input_characters_val.add(char)
    for char in target_text_val:
        if char not in target_characters_val:
            target_characters_val.add(char)

# num_encoder_tokens_val = len(input_characters_val)
# num_decoder_tokens_val = len(target_characters_val)
# max_encoder_seq_length_val = max([len(txt) for txt in input_texts_val]) + 1
# max_decoder_seq_length_val = max([len(txt) for txt in target_texts_val]) + 1
# print("Number of samples:", len(input_texts_val))
# print("Number of unique input tokens:", num_encoder_tokens_val)
# print("Number of unique output tokens:", num_decoder_tokens_val)
# print("Max sequence length for inputs:", max_encoder_seq_length_val)
# print("Max sequence length for outputs:", max_decoder_seq_length_val)

encoder_input_data_val = np.zeros((len(input_texts_val), max_encoder_seq_length), dtype="float32")
decoder_input_data_val = np.zeros((len(input_texts_val), max_decoder_seq_length), dtype="float32")
decoder_target_data_val = np.zeros((len(input_texts_val), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts_val, target_texts_val)):
    for t, char in enumerate(input_text):
        encoder_input_data_val[i, t] = input_token_index[char]
    encoder_input_data_val[i, t + 1:] =  input_token_index[" "]
    for t, char in enumerate(target_text):
        decoder_input_data_val[i, t] =  target_token_index[char]
        if t > 0:
            decoder_target_data_val[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data_val[i, t + 1 :] = target_token_index[" "]
    decoder_target_data_val[i, t:, target_token_index[" "]] = 1.0

In [ ]:
embedding_size = 128
batch_size = 128
latent_dim = 256
en_layer = 2
de_layer = 3
epochs = 100

In [ ]:
max_encoder_seq_length_word = max(input_texts, key = len)
max_decoder_seq_length_word = max(target_texts, key = len)
print(max_encoder_seq_length_word, max_decoder_seq_length_word)

 pradhaanaadhyaapakon   गैस्ट्रोइंटेस्टाइनल 


In [ ]:
#Callback for calculating word accuracy.

import numpy as np
class PredictionCallback(tf.keras.callbacks.Callback):
  def __init__(self, validation_data):
    self.validation_data = validation_data
  def on_epoch_end(self, epoch, logs={}):
    predictions = self.model.predict(self.validation_data[0])
    ground_truth = self.validation_data[1]
    one_hot_prediction = tf.one_hot(tf.argmax(predictions, axis=2), predictions.shape[2]).numpy()
    count = 0
    for i in range(len(predictions)):
      if np.array_equal(ground_truth[i], one_hot_prediction[i]):
        count+=1
    
    print("Word Level Val Accuracy: "+ str(count/len(predictions)))

In [ ]:
#GRU Model: Passing final state of encoder as the initial state of decoder.

encoder_inputs = Input(shape=(max_encoder_seq_length,))
print(encoder_inputs.shape)
encoder_input = Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
print(encoder_input.shape)
encoder = GRU(latent_dim, return_state=True, return_sequences=True)(encoder_input)
if en_layer == 2:
  encoder = GRU(latent_dim, return_state=True)(encoder)
if en_layer == 3:
  encoder = GRU(latent_dim, return_state=True, return_sequences=True)(encoder)
  encoder = GRU(latent_dim, return_state=True)(encoder)
encoder_outputs, state_h = encoder
decoder_inputs = Input(shape=(max_decoder_seq_length,))
decoder_input = Embedding(num_decoder_tokens, embedding_size)(decoder_inputs)
decoder = GRU(latent_dim, return_sequences=True)(decoder_input,initial_state=state_h)
if de_layer == 2:
  decoder = GRU(latent_dim, return_sequences=True)(decoder)
if de_layer == 3:
  decoder = GRU(latent_dim, return_sequences=True)(decoder)
  decoder = GRU(latent_dim, return_sequences=True)(decoder)
decoder_outputs = decoder
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

(None, 23)
(None, 23, 128)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 23)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 23, 128)      4480        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 22)]         0                                            
__________________________________________________________________________________________________
gru (GRU)                       [(None, 23, 256), (N 296448      embedding[0][0]                  
___________________________________________________________________

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
PC = PredictionCallback(validation_data= ([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val))
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    shuffle=True,
    validation_data=([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val),
    callbacks=([PC])
    )

In [ ]:
#LSTM Model: Passing the states of ith layer in encoder as initial states of ith layer in decoder.

Decoder_states = []
en = en_layer
en -= 1; de = en
encoder_inputs = keras.Input(shape = (max_encoder_seq_length,))
encoder_embedded = keras.layers.Embedding(input_dim = num_encoder_tokens, output_dim = embedding_size)(encoder_inputs)
encoder = keras.layers.LSTM(latent_dim, return_state = True, return_sequences = True)
outputs, state_h, state_c = encoder(encoder_embedded)
Decoder_states.append([state_h, state_c])
for i in range(en):
  en_lstm = keras.layers.LSTM(latent_dim, return_state = True, return_sequences = True)
  outputs, state_h, state_c = en_lstm(outputs)
  Decoder_states.append([state_h, state_c])

decoder_inputs = keras.Input(shape = (max_decoder_seq_length,))
decoder_embedded = keras.layers.Embedding(input_dim = num_decoder_tokens, output_dim = embedding_size)(decoder_inputs)
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences = True, return_state = True)
de_outputs, ignore1, ignore2 = decoder_lstm(decoder_embedded, initial_state = Decoder_states[0])
for i in range(de):
  de_lstm = keras.layers.LSTM(latent_dim, return_state = True, return_sequences = True)
  de_outputs, state_h, state_c = de_lstm(de_outputs, initial_state = Decoder_states[i+1])

# decoder_attention = keras.layers.AdditiveAttention(name = "decoder_attention")
# decoder_concat    = keras.layers.Concatenate(name = "decoder_concat")
# context_vec, attn_weights = decoder_attention([de_outputs, outputs], return_attention_scores = True)
# de_outputs = decoder_concat([de_outputs, context_vec])
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation = "softmax")
decoder_outputs = decoder_dense(de_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
PC = PredictionCallback(validation_data = ([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val))
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size = batch_size,
    epochs = epochs,
    shuffle = True,
    validation_data = ([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val),
    callbacks = ([PC])
    )

## **Code with Wandb Sweeps**

---



In [ ]:
!pip install wandb -qqq

     |████████████████████████████████| 1.8MB 5.5MB/s 
     |████████████████████████████████| 133kB 23.3MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 174kB 22.1MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 


In [ ]:
import wandb
wandb.login()
from wandb.keras import WandbCallback as WC
import warnings
warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_config = {
    'method' : 'bayes',
    'metric' :
    {
        'name' : 'word_validation_accuracy',
        'goal' : 'maximize'
    },
    'parameters' :
    {
        'epochs' :
        {
            'values' : [20, 30, 40]
        },
        'batch_size' :
        {
            'values' : [64, 128, 256]
        },
        'embedding_size' :
        {
            'values' : [64, 128, 256]
        },
        'cell_type' :
        {
            'values' : ["lstm", "gru", "vanillaRNN"],
        },
        'learning_rate' :
        {
            'values' : [1e-3, 2e-3, 3e-3]
        },
        'optimizer' : {
            'values' : ['rmsprop', 'adam']
        },
        'encoder_layers' :
        {
            'values' : [1, 2, 3]
        },
        'hidden_layer_size' :
        {
            'values' : [128, 256]
        },
        'dropout' :
        {
            'values' : [0, 0.2, 0.3, 0.5]
        },
        'attention' :
        {
            'values' : ['no', 'yes']
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="cs6910-assignment-3")

Create sweep with ID: 0aurqg24
Sweep URL: https://wandb.ai/priyanka/cs6910-assignment-3/sweeps/0aurqg24


In [ ]:
max_encoder_seq_length = 0
max_decoder_seq_length = 0
num_encoder_tokens     = 0
num_decoder_tokens     = 0
input_token_index      = {}
arget_token_index      = {}
input_characters       = set()
target_characters      = set()
def Vectorize_Data(Data, isTrain):

  global input_characters, target_characters,  num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index
  input_texts       = []
  target_texts      = []
  input_characters  = set()
  target_characters = set()
  for index, line in Data.iterrows():
      input_text, target_text = line['English'], line['Hindi']
      input_text, target_text = str(input_text), str(target_text)
      target_text = " " + target_text + " "
      input_text  = " " + input_text + " "
      input_texts.append(input_text)
      target_texts.append(target_text)
      if isTrain:
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
  if isTrain:
    input_characters       = sorted(list(input_characters))
    target_characters      = sorted(list(target_characters))
    num_encoder_tokens     = len(input_characters)
    num_decoder_tokens     = len(target_characters)
    max_encoder_seq_length = max([len(txt) for txt in input_texts])+1
    max_decoder_seq_length = max([len(txt) for txt in target_texts])+1

    input_token_index  = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

  encoder_input_data  = np.zeros((len(input_texts), max_encoder_seq_length), dtype = "float32")
  decoder_input_data  = np.zeros((len(input_texts), max_decoder_seq_length), dtype = "float32")
  decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = "float32")

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t]   =  input_token_index[char]
      encoder_input_data[i, t + 1 :] = input_token_index[" "]
      for t, char in enumerate(target_text):
          decoder_input_data[i, t]   = target_token_index[char]
          if t > 0:
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :] = target_token_index[" "]
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0

  return input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data

In [ ]:
def RNN():

  input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data = Vectorize_Data(train, 1)
  input_texts_val, target_texts_val, encoder_input_data_val, decoder_input_data_val, decoder_target_data_val = Vectorize_Data(val, 0)
  input_texts_test, target_texts_test, encoder_input_data_test, decoder_input_data_test, decoder_target_data_test = Vectorize_Data(test, 0)

  config_defaults = {
      'epochs'            : 20,
      'batch_size'        : 128,
      'embedding_size'    : 32,
      'cell_type'         : "gru",
      'learning_rate'     : 3e-3,
      'optimizer'         : "adam",
      'encoder_layers'    : 3,
      'decoder_layers'    : 3,
      'hidden_layer_size' : 256,
      'dropout'           : 0,
      'attention'         : 'yes'
  }

  wandb.init(project= 'cs6910-assignment-3', config = config_defaults)
  config            = wandb.config
  name              = "CELL=" + str(config.cell_type)  + "_OP=" + str(config.optimizer) + "_BS=" + str(config.batch_size) + "_D=" + str(config.dropout)
  wandb.init().name = name
  latent_dim        = config.hidden_layer_size
  embedding_size    = config.embedding_size
  en_layer          = config.encoder_layers
  #de_layer          = config.decoder_layers
  lr                = config.learning_rate
  batch_size        = config.batch_size
  cell              = config.cell_type
  optimizer         = config.optimizer
  drop              = config.dropout
  epochs            = config.epochs
  attention         = config.attention

  #Model

  global Decoder_states, word_val_acc
  Decoder_states = []
  en_layer -= 1
  de_layer  = en_layer
  encoder_input     = Input(shape=(max_encoder_seq_length,))
  encoder_embedding = Embedding(input_dim = num_encoder_tokens, output_dim = embedding_size)(encoder_input)
  
  if cell == "vanillaRNN":
    encoder = SimpleRNN(latent_dim, return_state = True, return_sequences = True)
    outputs, state_h = encoder(encoder_embedding)
    Decoder_states.append([state_h])
    for i in range(en_layer):
      encoder_rnn = SimpleRNN(latent_dim, return_state = True, return_sequences = True, dropout=drop)
      outputs, state_h = encoder_rnn(outputs)
      Decoder_states.append([state_h])

  elif cell == "lstm":
    encoder = LSTM(latent_dim, return_state = True, return_sequences = True)
    outputs, state_h, state_c = encoder(encoder_embedding)
    Decoder_states.append([state_h, state_c])
    for i in range(en_layer):
      encoder_lstm = keras.layers.LSTM(latent_dim, return_state = True, return_sequences = True, dropout = drop)
      outputs, state_h, state_c = encoder_lstm(outputs)
      Decoder_states.append([state_h, state_c])

  elif cell == "gru":
    encoder = GRU(latent_dim, return_state = True, return_sequences = True)
    outputs, state_h = encoder(encoder_embedding)
    Decoder_states.append([state_h])
    for i in range(en_layer):
      encoder_gru =  GRU(latent_dim, return_state = True, return_sequences = True, dropout = drop)
      outputs, state_h = encoder_gru(outputs)
      Decoder_states.append([state_h])

  decoder_input = Input(shape=(max_decoder_seq_length,))
  decoder_embedding = Embedding(input_dim = num_decoder_tokens, output_dim = embedding_size)(decoder_input)
  
  if cell == "vanillaRNN":
    decoder = SimpleRNN(latent_dim, return_state = True, return_sequences = True)
    de_outputs, _ = decoder(decoder_embedding, initial_state = Decoder_states[0])
    for i in range(de_layer):
       decoder_rnn = SimpleRNN(latent_dim, return_state = True, return_sequences = True, dropout = drop)
       de_outputs, state_h = decoder_rnn(de_outputs, initial_state = Decoder_states[i+1])

  elif cell == "lstm":
    decoder = LSTM(latent_dim, return_state = True, return_sequences = True)
    de_outputs, _, _ = decoder(decoder_embedding, initial_state = Decoder_states[0])
    for i in range(de_layer):
      decoder_lstm = LSTM(latent_dim, return_state = True, return_sequences = True, dropout = drop)
      de_outputs, state_h, state_c = decoder_lstm(de_outputs, initial_state = Decoder_states[i+1])

  elif cell == "gru":
    decoder = GRU(latent_dim, return_state = True, return_sequences = True)
    de_outputs, _ = decoder(decoder_embedding, initial_state = Decoder_states[0])
    for i in range(de_layer):
      decoder_gru = GRU(latent_dim, return_state = True, return_sequences = True, dropout = drop)
      de_outputs, state_h = decoder_gru(de_outputs, initial_state = Decoder_states[i+1])

  if attention == "yes":
      decoder_attention = keras.layers.AdditiveAttention(name = "decoder_attention")
      decoder_concat    = keras.layers.Concatenate(name = "decoder_concat")
      context_vec, attn_weights = decoder_attention([de_outputs, outputs], return_attention_scores = True)
      de_outputs = decoder_concat([de_outputs, context_vec])

  decoder_dense = Dense(num_decoder_tokens, activation = "softmax")
  decoder_outputs = decoder_dense(de_outputs)

  #Callback
  class PredictionCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data):
      self.validation_data = validation_data
    def on_epoch_end(self, epoch, logs = {}):
      predictions = self.model.predict(self.validation_data[0])
      ground_truth = self.validation_data[1]
      one_hot_prediction = tf.one_hot(tf.argmax(predictions, axis=2), predictions.shape[2]).numpy()
      count = 0
      for i in range(len(predictions)):
        if np.array_equal(ground_truth[i], one_hot_prediction[i]):
          count += 1
      print("Word Level Val Accuracy: " + str(count/len(predictions)))
      word_val_acc.append(count/len(predictions))

  model = keras.Model([encoder_input, decoder_input], decoder_outputs)
  word_val_acc = []
  if optimizer == "adam":
    opt = keras.optimizers.Adam(learning_rate = lr)
  if optimizer == "rmsprop":
    opt = keras.optimizers.RMSprop(learning_rate = lr)
  model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])
  
  PC = PredictionCallback(validation_data = ([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val))
  hist = model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size = batch_size,
        epochs = epochs,
        shuffle = True,
        validation_data = ([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val),
        callbacks =([PC]))
  
  
  val_acc = hist.history['val_accuracy']
  val_loss = hist.history['val_loss']
  acc = hist.history['accuracy']
  loss = hist.history['loss']

  for i in range(epochs):
    wandb.log({'word_validation_accuracy': word_val_acc[i]})
    wandb.log({'validation_accuracy': val_acc[i]})
    wandb.log({'validation_loss': val_loss[i]})
    wandb.log({'training_accuracy': acc[i]})
    wandb.log({'training_loss': loss[i]})

In [ ]:
wandb.agent('euzuj3p6', function = RNN)

wandb: Agent Starting Run: lcz6wwh4 with config:
wandb: 	attention: no
wandb: 	batch_size: 256
wandb: 	cell_type: gru
wandb: 	dropout: 0
wandb: 	embedding_size: 256
wandb: 	encoder_layers: 2
wandb: 	epochs: 20
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.001
wandb: 	optimizer: rmsprop
wandb: Currently logged in as: priyanka (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/20
173/173 [==============================] - 21s 82ms/step - loss: 1.3348 - accuracy: 0.7060 - val_loss: 0.8204 - val_accuracy: 0.7773
Word Level Val Accuracy: 0.0004589261128958238
Epoch 2/20
173/173 [==============================] - 13s 73ms/step - loss: 0.7807 - accuracy: 0.7855 - val_loss: 0.5993 - val_accuracy: 0.8250
Word Level Val Accuracy: 0.0016062413951353832
Epoch 3/20
173/173 [==============================] - 13s 73ms/step - loss: 0.5400 - accuracy: 0.8421 - val_loss: 0.3824 - val_accuracy: 0.8828
Word Level Val Accuracy: 0.05392381826525929
Epoch 4/20
173/173 [==============================] - 13s 73ms/step - loss: 0.3826 - accuracy: 0.8829 - val_loss: 0.3052 - val_accuracy: 0.9066
Word Level Val Accuracy: 0.11519045433685177
Epoch 5/20
173/173 [==============================] - 13s 73ms/step - loss: 0.2966 - accuracy: 0.9085 - val_loss: 0.2761 - val_accuracy: 0.9145
Word Level Val Accuracy: 0.1466268930702157
Epoch 6/20
173/173 [==============================] 

word_validation_accuracy,0.31735
_runtime,295
_timestamp,1621267730
_step,99
validation_accuracy,0.94951
validation_loss,0.19427
training_accuracy,0.99113
training_loss,0.02944


word_validation_accuracy,▁▁▂▃▄▆▆▇▇▇▇███▇█▇███
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
validation_accuracy,▁▃▅▆▇▇██████████████
validation_loss,█▆▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_accuracy,▁▃▄▅▆▆▇▇▇▇▇█████████
training_loss,█▆▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: cmvdfqme with config:
wandb: 	attention: yes
wandb: 	batch_size: 128
wandb: 	cell_type: lstm
wandb: 	dropout: 0.2
wandb: 	embedding_size: 128
wandb: 	encoder_layers: 3
wandb: 	epochs: 30
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.001
wandb: 	optimizer: rmsprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

Epoch 1/30
691/691 [==============================] - 59s 72ms/step - loss: 1.0493 - accuracy: 0.7462 - val_loss: 0.3689 - val_accuracy: 0.8899
Word Level Val Accuracy: 0.06080770995869665
Epoch 2/30
691/691 [==============================] - 46s 66ms/step - loss: 0.3416 - accuracy: 0.8951 - val_loss: 0.2023 - val_accuracy: 0.9371
Word Level Val Accuracy: 0.26525929325378617
Epoch 3/30
691/691 [==============================] - 46s 66ms/step - loss: 0.2102 - accuracy: 0.9335 - val_loss: 0.1777 - val_accuracy: 0.9447
Word Level Val Accuracy: 0.3012849931161083
Epoch 4/30
691/691 [==============================] - 46s 67ms/step - loss: 0.1745 - accuracy: 0.9443 - val_loss: 0.1648 - val_accuracy: 0.9484
Word Level Val Accuracy: 0.3260670032124828
Epoch 5/30
691/691 [==============================] - 48s 69ms/step - loss: 0.1554 - accuracy: 0.9501 - val_loss: 0.1615 - val_accuracy: 0.9496
Word Level Val Accuracy: 0.3432767324460762
Epoch 6/30
691/691 [==============================] - 45s 

word_validation_accuracy,0.35957
_runtime,1479
_timestamp,1621280650
_step,149
validation_accuracy,0.95322
validation_loss,0.16033
training_accuracy,0.96298
training_loss,0.11175


word_validation_accuracy,▁▆▆▇▇██▇▇▇████████▇███████▇███
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
validation_accuracy,▁▆▇▇▇█████████████████████████
validation_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_accuracy,▁▆▇▇▇█████████████████████████
training_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: ohe8buys with config:
wandb: 	attention: no
wandb: 	batch_size: 128
wandb: 	cell_type: gru
wandb: 	dropout: 0.5
wandb: 	embedding_size: 64
wandb: 	encoder_layers: 3
wandb: 	epochs: 30
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.002
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/30
346/346 [==============================] - 33s 71ms/step - loss: 1.3049 - accuracy: 0.7195 - val_loss: 0.7940 - val_accuracy: 0.7838
Word Level Val Accuracy: 0.0002294630564479119
Epoch 2/30
346/346 [==============================] - 23s 66ms/step - loss: 0.7462 - accuracy: 0.7922 - val_loss: 0.5193 - val_accuracy: 0.8469
Word Level Val Accuracy: 0.008949059201468563
Epoch 3/30
346/346 [==============================] - 23s 66ms/step - loss: 0.5248 - accuracy: 0.8446 - val_loss: 0.3571 - val_accuracy: 0.8942
Word Level Val Accuracy: 0.07893529141808169
Epoch 4/30
346/346 [==============================] - 23s 66ms/step - loss: 0.3766 - accuracy: 0.8862 - val_loss: 0.2624 - val_accuracy: 0.9199
Word Level Val Accuracy: 0.17829279486002753
Epoch 5/30
346/346 [==============================] - 23s 66ms/step - loss: 0.2954 - accuracy: 0.9084 - val_loss: 0.2284 - val_accuracy: 0.9295
Word Level Val Accuracy: 0.22372648003671408
Epoch 6/30
346/346 [==============================] 